### **RunnableBranch**

- 입력에 따라 동적으로 로직을 라우팅 할 수 있는 도구 
    - 입력 데이터의 특성에 기반하여 다양한 처리 경로를 유연하게 정의 

- 복잡한 의사 결정 트리를 간단하고 직관적인 방식으로 구현 
    - 코드의 가독성과 유지보수성을 향상, 로직의 모듈화와 재사용성을 촉진 

### **입력에 따른 동적 로직 라우팅**

- 라우팅을 통해 이전 단계의 출력이 다음 단계를 정의하는 비결정적 체인 생성

1. `RunnableLambda`에서 조건부로 실행 가능한 객체를 반환

2. `RunnableBranch` 사용 

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

### **RunnableLambda를 사용한 Routing**

In [ ]:
# 간단한 분류 예시
prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

llm = ChatOpenAI(model="gpt-4o-mini")

chain = (prompt | llm | StrOutputParser())

# 수학 질문
print(chain.invoke({"question": "2+2 는 무엇인가요?"}))

# 과학 질문
print(chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"}))

# 기타 질문
chain.invoke({"question": "Google은 어떤 회사인가요?"})

### **여러가지 하위 체인 생성**

In [18]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | llm
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | llm
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    | llm
)

### **사용자 함수 정의**

In [8]:
def route(info):
    
    # 주제에 수학이 포함되어 있는지 확인
    if "수학" in info["topic"].lower():
        return math_chain
    
    # 주제에 과학이 포함되어 있는지 확인
    elif "과학" in info["topic"].lower():
        return science_chain
    
    # 그 외의 경우 general_chain을 사용
    else:
        return general_chain

In [12]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

full_chain = (
    {"topic": chain, "question": RunnablePassthrough()}
    | RunnableLambda(route)
    | StrOutputParser()
)

print(full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."}))

print(full_chain.invoke({"question": "중력은 어떻게 작용하나요?"}))

깨봉선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 주로 변화와 형태를 다루는 데 중점을 둡니다. 미적분학은 두 가지 주요 부분으로 나뉘어 있습니다: 미분과 적분입니다.

미분은 함수의 변화율을 다루며, 특정한 점에서의 기울기를 구하는 과정이라고 할 수 있습니다. 예를 들어, 자동차의 속도를 측정할 때, 특정 시간에서의 순간 속도를 구하는 것이 미분의 예입니다.

반면, 적분은 어떤 함수의 면적을 구하는 과정으로, 주로 곡선 아래의 영역을 계산하는 데 사용됩니다. 만약 자동차가 이동한 거리나 물체의 부피를 구하는 것이 필요할 때 적분을 사용합니다.

미적분학은 물리학, 공학, 경제학 등 여러 분야에서 필수적으로 사용되며, 변화와 관련된 문제를 푸는 데 매우 유용한 도구입니다.
아이작 뉴턴 선생님께서 말씀하시기를, 중력은 두 물체 사이의 인력으로, 물체의 질량과 거리에 따라 달라진다고 하셨습니다. 그 법칙은 즉, 두 물체의 질량이 클수록 중력이 강해지고, 두 물체 사이의 거리가 멀어질수록 중력이 약해진다는 것입니다. 이는 그의 중력 법칙으로 설명되며, 모든 물체가 서로 끌어당기는 힘을 가지고 있다는 사실을 강조합니다. 따라서, 우리가 지구 위에 서 있을 때, 지구가 우리를 끌어당기는 힘이 바로 중력인 것입니다.


### **RunnableBranch를 사용한 Routing**

- 입력값에 따라 실행 조건과 `Runnable`을 정의 할 수 있는 특별한 `Runnable`

문법
- `RunnableBranch`는 조건쌍의 리스트와 기본 Runnable로 초기화
- 호출 시 입력값을 각 조건에 전달하여 분기 선택
- True로 평가되는 첫 번째 조건 선택, 조건에 해당하는 Runnable을 입력값과 함께 실행
- 일치하는 조건이 없으면 기본 `Runnable`을 실행

In [21]:
from langchain_core.runnables import RunnableBranch
from operator import itemgetter

branch = RunnableBranch(
    
    (lambda x: '수학' in x['topic'].lower(), math_chain),
    (lambda x: '과학' in x['topic'].lower(), science_chain),
    general_chain
    )

full_chain = (
    {"topic": chain, "question": RunnablePassthrough()}
    | branch
    | StrOutputParser()
)

full_chain.invoke("미적분의 개념에 대해 말씀해 주세요.")

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 함수의 변화와 경향을 다루는 중요한 도구입니다. 미적분은 크게 두 가지 주요 개념인 미분과 적분으로 나뉩니다. \n\n미분은 함수의 기울기를 구하거나 변화율을 측정하는 과정이며, 주어진 점에서 함수가 어떻게 변화하는지를 분석합니다. 예를 들어, 물체의 속도의 변화를 분석할 때 미분을 사용합니다.\n\n반면에 적분은 함수를 특정 구간에서의 총합이나 면적을 구하는 과정입니다. 적분은 주어진 함수 아래의 면적을 계산하고, 데이터의 총합을 구하는 데 많이 활용됩니다. \n\n미적분은 자연 현상, 경제학, 공학 등 다양한 분야에서 활용되며, 문제를 해결하는 데 없어서는 안 될 중요한 도구로 자리잡고 있습니다.'